# Project: Conway's Game of Life - CPU vs GPU Implementation 

In this project, we are going to see implementations of **Conway's Game of Life**, a classic cellular automaton in three ways: a pure python approach (to run on the CPU), a vectorised approach using NumPy (to run on the CPU) and then using CuPy (to run on the GPU). We'll also visualise the evolution of the Game of Life grid to see the computation in action. 

## What is Conway's Game of Life?
It's a zero-player game devised by John Conway, where you have a grid of cells that live or die based on a few simple rules:
- Each cell can be "alive" (1) or "dead" (0).
- At each time step (generation), the following rules apply to every cell simultaneously:
- Any live cell with fewer than 2 live neighbours dies (underpopulation).
- Any live cell with 2 or 3 live neighbours lives on to the next generation (survival).
- Any live cell with more than 3 live neighbours dies (overpopulation).
- Any dead cell with exactly 3 live neighbours becomes a live cell (reproduction).
- Neighbours are the 8 cells touching a given cell horizontally, vertically, or diagonally.
- From these simple rules emerges a lot of interesting behaviour – stable patterns, oscillators, spaceships (patterns that move), etc. It's a good example of a grid-based simulation that can benefit from parallel computation because the state of each cell for the next generation can be computed independently (based on the current generation).

## Visualisation of Game of Life
To make this project more visually engaging, below is an **animated GIF** showing an example of a Game of Life simulation starting from a random initial configuration. White pixels represent live cells, and black pixels represent dead cells. You can see patterns forming, moving, and changing over time:
An example evolution of Conway's Game of Life over a few generations (white = alive, black = dead).
The animation demonstrates how random initial clusters of cells can evolve into interesting patterns. Notice some cells blink on and off or form moving patterns.

The animation shows 50 timesteps on a 100x100 grid. 

![Conways Game of Life](../_static/game_of_life.gif)


## Implementations
All of the implementation for the three different versions (Pure Python, NumPy and CuPy) are contained within the `.py` located at `content/game_of_life.py`. 

To run the different versions of the code, you can use:

**Naïve Python Version**

`python game_of_life.py run_life_naive --size 100 --timesteps 50`
which will produce a file called `game_of_life_naive.gif`.

**CPU-Vectorized Version**
`python game_of_life.py run_life_numpy --size 100 --timesteps 50`
which will produce a file called `game_of_life_cpu.gif`.

**GPU-Accelerated Version**
`python game_of_life.py run_life_cupy --size 100 --timesteps 50`
which will produce a file called `game_of_life_gpu.gif`.

## Naive Implementation
The core computation that is being performed for the naive implementation is: 
```python
def life_step_naive(grid: np.ndarray) -> np.ndarray:
    N, M = grid.shape
    new = np.zeros((N, M), dtype=int)
    for i in range(N):
        for j in range(M):
            cnt = 0
            for di in (-1, 0, 1):
                for dj in (-1, 0, 1):
                    if di == 0 and dj == 0:
                        continue
                    ni, nj = (i + di) % N, (j + dj) % M
                    cnt += grid[ni, nj]
            if grid[i, j] == 1:
                new[i, j] = 1 if (cnt == 2 or cnt == 3) else 0
            else:
                new[i, j] = 1 if (cnt == 3) else 0
    return new

def simulate_life_naive(N: int, timesteps: int, p_alive: float = 0.2):
    grid = np.random.choice([0, 1], size=(N, N), p=[1-p_alive, p_alive])
    history = []
    for _ in range(timesteps):
        history.append(grid.copy())
        grid = life_step_naive(grid)
    return history
```
### Explanation 

There are a number of different reasons that the naive implementation runs slow, including: 
- **Nested Python Loops**: Instead of eight `np.roll` calls and one `np.where`, we make two loops over `i, j` (10^4 iterations) and two more loops over `di, dj` (9 checks each), for roughly 9x10^4 Python level operation per step. 
- **Manual edge-wrapping logic**: Branching (`if ni < 0 … elif …`) for each neighbour check, instead of the single fast shift that `np.roll` does in C. 
- **Per-cell rule application** The game of life rule is applied with Python `if/else` instead of the single vectorised Boolean mask. 
- **Rebuilding a new NumPy array element-by-element**: writing into `new_grid[i, j]` in Python is orders of magnitude slower than one-shot `np.where`. 

Together, these overheads make this version run very slow, particularly as `N` begins to increase, and would not leverage any low-level C loops or GPU acceleration. 

## CPU-Vectorised Implementation 

```python
def life_step_numpy(grid: np.ndarray) -> np.ndarray:
    neighbours = (
        np.roll(np.roll(grid, 1, axis=0), 1, axis=1) +
        np.roll(np.roll(grid, 1, axis=0), -1, axis=1) +
        np.roll(np.roll(grid, -1, axis=0), 1, axis=1) +
        np.roll(np.roll(grid, -1, axis=0), -1, axis=1) +
        np.roll(grid, 1, axis=0) +
        np.roll(grid, -1, axis=0) +
        np.roll(grid, 1, axis=1) +
        np.roll(grid, -1, axis=1)
    )
    return np.where((neighbours == 3) | ((grid == 1) & (neighbours == 2)), 1, 0)


def simulate_life_numpy(N: int, timesteps: int, p_alive: float = 0.2):
    grid = np.random.choice([0, 1], size=(N, N), p=[1-p_alive, p_alive])
    history = []
    for _ in range(timesteps):
        history.append(grid.copy())
        grid = life_step_numpy(grid)
    return history
```

### Explanation

#### From Per-Cell Loops to Whole-Array Operations 
In the **naive** version, every one of the NxN cells in Python was traversed within two nested loops; then, for each cell, two more loops over the offsets `di` and `dj` counted its eight neighbours by computing. `(i + di) % N` and `(j + dj) % M` in pure Python. 
**Cost**: ~9·N² Python-level iterations per generation, including branching and modulo arithmetic.
**Drawback** Thousands of interpreter calls and non-contiguous memory access. 
In the **NumPy** version, no Python loops over individual cells occur. Instead, eight calls to `np.roll` shift the entire grid array (up, down, left, right and on diagonals), automatically handling wrap-around in one C-level operation. Summing those eight arrays gives a full neighbour count in a single, optimised pass. 

#### Manual `if/else` vs Vectorised Mask 
In the **naive** implementation, after counting neighbours, each cell's fate is determined with a Python `if grid[i,j] == 1: ... else: ...` and assigned via `new[i,j] = ...`. 
In the **NumPy** implementation a single expression of `(neighbours == 3) | ((grid == 1) & (neighbours == 2))` produces an NxN Boolean mask of *cells alive next*. Converting that mask to integers with `np.where(mask, 1, 0)` builds the entire next-generation grid in one C-level operation, resulting in no per-element Python overhead. 


#### Automatic Wrap-Around vs Manual Modulo Logic
In the **naive** version, every neighbour checks does: 

```python 
ni = (i + di) % N
nj = (j + dj) % M
```

with Python-level branching and modulo arithmetic on each of the 9 checks per cell. The associated **cost** is thousands of `%` operations and branch instructions per generation. 

In the **NumPy** version, a single call to 

```python
np.roll(grid, shift, axis=)
```
automatically wraps the entire array in one C-level operation. The **benefit** is that all per-cell `%` operations and branching are eliminated, being replaced by a single optimised memory shift over the whole grid. 

## GPU-Accelerated Implementation 

```python
def life_step_gpu(grid: cp.ndarray) -> cp.ndarray:
    neighbours = (
        cp.roll(cp.roll(grid, 1, axis=0), 1, axis=1) +
        cp.roll(cp.roll(grid, 1, axis=0), -1, axis=1) +
        cp.roll(cp.roll(grid, -1, axis=0), 1, axis=1) +
        cp.roll(cp.roll(grid, -1, axis=0), -1, axis=1) +
        cp.roll(grid, 1, axis=0) +
        cp.roll(grid, -1, axis=0) +
        cp.roll(grid, 1, axis=1) +
        cp.roll(grid, -1, axis=1)
    )
    return cp.where((neighbours == 3) | ((grid == 1) & (neighbours == 2)), 1, 0)


def simulate_life_cupy(N: int, timesteps: int, p_alive: float = 0.2):
    grid_gpu = (cp.random.random((N, N)) < p_alive).astype(cp.int32)
    history = []
    for _ in range(timesteps):
        history.append(cp.asnumpy(grid_gpu))
        grid_gpu = life_step_gpu(grid_gpu)
    return history

```

### CuPy vs NumPy: What's Changed.

The power of **CuPy** lies in its near drop-in compatibility with **NumPy**: arrays live on the GPU, and computations run in parallel on the Device, yet the code looks almost identical. 

#### Imports 

The first change you will need to make is to use CuPy rather than NumPy. 
**NumPy**: 
```Python 
import numpy as np
```

**CuPy**: 
```Python 
import cupy as cp
```

#### Random initialisation 
**NumPy**: 
```Python 
grid = np.random.choice([0,1], size=(N,N), p=[1-p, p])
```

**CuPy**: 
```Python 
grid_gpu = (cp.random.random((N,N)) < p_alive).astype(cp.int32)
```

#### Data Transfer
**CuPy**: 
```Python 
cp.asnumpy(grid_gpu)  # bring a CuPy array back to NumPy
```

### Which to use?
**Large grids (e.g. N ≥ 500) or many timesteps**: GPU's parallel throughput outweighs kernel-launch and transfer overhead.
**Small grids (e.g. 10×10)**: GPU overhead may dominate, so you may want to stick with NumPy.

### Why is this quicker?

When a computation can be expressed as the same operation applied independently across many data elements, like counting neighbours on every cell of a large Game of Life grid, GPUs often deliver dramatic speedups compared to CPUs. This advantage stems from several architectural and compiler-related factors that we discussed earlier in the section on theory, including: 
- **Massive Data Parallelism**
    - **CPU**: A few (4–16) powerful cores optimised for sequential tasks and complex control flow.
    - **GPU**: Hundreds to thousands of simpler cores running in lock-step.
    - **Result**: A Game of Life update, which is identical work on each of N² cells, can be dispatched as thousands of parallel GPU threads, sweeping through the grid in a single kernel launch instead of looping in software.
- **Throughput-Oriented Design**
    - **CPU cores** focus on single-thread performance (high clock speeds, deep pipelines, branch prediction).
    - **GPU cores** sacrifice single-thread speed in favour of raw arithmetic throughput and memory bandwidth across many threads.
    - **Result**: When you need to process millions of cell updates, the GPU's aggregate arithmetic and memory bandwidth far outstrips what a CPU can deliver.
- **Specialised Memory Hierarchy**
    - **CPU**: Large multi-level caches and direct access to system RAM.
    - **GPU:** High-bandwidth device memory (VRAM), with its own caches and shared memory for thread blocks.
    - **Result**: Once the grid is transferred to GPU memory, all subsequent neighbour-count rolls and mask evaluations occur on-device, benefiting from coalesced global reads and fast on-chip scratchpads.
- **Compiled GPU Kernels vs. Interpreted Loops**
    - **CPU code** that uses Python loops incurs per-iteration interpreter overhead. Even NumPy's C loops run on a single core.
    - **GPU kernels** compiled ahead of time by NVCC or generated at runtime execute the same inner logic entirely in device code without returning to Python between elements.
    - **Result**: You replace thousands of Python bytecode dispatches or even C-loop iterations with just a few kernel launches and a handful of device-resident function calls.

**In summary,**, for problems like Conway's Game of Life, where the same simple computation is applied independently across a large array, GPUs excel by running thousands of data-parallel threads in hardware, backed by specialised memory systems and aggressive compiler optimisations. Offloading to the GPU transforms an O(N²) loop of Python or C iterations into just a handful of highly parallel kernel launches, yielding orders-of-magnitude speedups on sufficiently large grids.

### How much quicker?
Each implementation exhibits a different overall runtime, as you have probably noticed when running them from the command line. We can use the built-in UNIX command line tool `time` to measure the time that is taken to run the code. 

Modify it so that the exercise at the end of conways game of life instead use just time python to get the overall run time that can then be used maybe with the poetry run command and then use that for the scaling plot
---- Include a scaling graph here for the differnet expeirments that have been done, this should be based on my personal machine, eg.. an RTX 3070 ----

### Finer Grained Timing Information
Our next step is to quantify those differences by measuring exactly how long each stage takes (pure computation, data transfers, grid initialisation, etc.) and to pinpoint where the bulk of the time is spent. The following section will address these questions by introducing profiling techniques.